# Altinha Ball Tracking and Hit Detection

This notebook performs ball tracking and hit detection on altinha (Brazilian footvolley) videos using YOLO object detection.

<a href="https://colab.research.google.com/github/kifjj/altinha-play/blob/main/alta_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

Install required packages with specific versions for compatibility.


In [1]:
!pip install "numpy<2.0" "scipy<1.14" supervision ultralytics "opencv-python-headless<4.12"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 51.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 37.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s et

## Import Libraries


In [2]:
import json
import cv2
import numpy as np
import supervision as sv
from typing import Dict, List, Optional, Sequence, Tuple
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Configuration

Set up paths and hyperparameters for ball detection and hit counting.


In [3]:
# Video and model paths
VIDEO_PATH = '/kaggle/input/alta-videos/altinha-beach-green-mq-13s.mp4'
MODEL_PATH = '/kaggle/input/yolo-ft-2511/pytorch/default/1/altinha_best.pt'
OUTPUT_PATH = '/kaggle/working/altinha-beach-green-mq-BEST_ONLY.mp4'

POSE_MODEL_PATH = '/kaggle/input/yolo11-pose/pytorch/default/1/yolo11l-pose.pt'

# Debug output directory for hit frames
DEBUG_FRAMES_DIR = '/kaggle/working/debug_frames'

# Detection parameters
CONFIDENCE_THRESHOLD = 0.05  # Minimum confidence for initial detection
MIN_CONFIDENCE = 0.08  # Minimum confidence to keep a detection
IOU_NMS = 0.5  # NMS IoU threshold

# Hit detection parameters
MIN_VERTICAL_AMPLITUDE = 3  # Minimum pixels for a valid hit (vertical movement)
MIN_FRAMES_BETWEEN_HITS = 8  # Minimum frames between consecutive hits
GAP_RESET_FRAMES = 30  # Frames without detection before resetting trajectory (1 sec at 30fps)


## Initialize Models and Annotators


In [4]:
import shutil
import os

# Copy pose model to writable directory to avoid read-only file system error
pose_model_writable = '/kaggle/working/yolo11l-pose.pt'
if not os.path.exists(pose_model_writable):
    shutil.copy(POSE_MODEL_PATH, pose_model_writable)

# Load YOLO models
model = YOLO(MODEL_PATH)
model_pose = YOLO(pose_model_writable)  # Load from writable location

# Setup annotators
box_annotator = sv.BoxAnnotator(
    thickness=2,
    color=sv.Color.from_hex("#00FF00")
)

label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_thickness=2,
    text_position=sv.Position.TOP_CENTER,
)

## Helper Functions


In [5]:
from typing import Dict, List, Optional, Sequence, Tuple

import numpy as np
import supervision as sv
from ultralytics import YOLO

BallPosition = Tuple[int, float, float, np.ndarray]
HitDetections = List[Dict[str, object]]


def filter_best_ball_detection(detections: sv.Detections, min_confidence: float) -> sv.Detections:
    """
    Filter detections to keep only the best one (highest confidence).
    
    Args:
        detections: sv.Detections object
        min_confidence: Minimum confidence threshold
        
    Returns:
        Filtered sv.Detections object (empty if no valid detection)
    """
    if len(detections) == 0:
        return detections
    
    best_idx = int(np.argmax(detections.confidence))
    best_conf = float(detections.confidence[best_idx])
    
    if best_conf >= min_confidence:
        return detections[best_idx:best_idx+1]
    
    return detections[0:0]  # Return empty detections


def update_ball_tracking_state(
    detections: sv.Detections,
    n_frame: int,
    last_ball_positions: List[BallPosition],
    last_ball_detection_n_frame: Optional[int],
    gap_reset_frames: int,
) -> Tuple[List[BallPosition], Optional[int]]:
    """
    Update ball tracking state with new detection.
    
    Args:
        detections: sv.Detections object
        n_frame: Current frame number
        last_ball_positions: List of (frame_idx, x_center, y_center, bbox) tuples
        last_ball_detection_n_frame: Last frame with detection
        gap_reset_frames: Max gap before resetting trajectory
        
    Returns:
        tuple: (updated_last_positions, updated_last_detection_frame)
    """
    if len(detections) == 0:
        return last_ball_positions, last_ball_detection_n_frame
    
    # Check for gap in detections
    if last_ball_detection_n_frame is not None:
        gap = n_frame - last_ball_detection_n_frame
        if gap > gap_reset_frames:
            last_ball_positions = []
    
    # Get ball center and bbox
    bbox = detections.xyxy[0]
    x1, y1, x2, y2 = bbox.tolist()
    x_center = 0.5 * (x1 + x2)
    y_center = 0.5 * (y1 + y2)
    
    # Update position history (keep last 3)
    last_ball_positions = last_ball_positions.copy()
    last_ball_positions.append((n_frame, x_center, y_center, bbox))
    if len(last_ball_positions) > 3:
        last_ball_positions.pop(0)
    
    return last_ball_positions, n_frame


def detect_hit(last_positions: List[BallPosition]) -> Tuple[bool, Optional[int], Optional[float], Optional[float], Optional[np.ndarray]]:
    """
    Detect if a hit occurred based on ball trajectory.
    A hit is detected when the ball reaches a local maximum in y-coordinate (bottom of screen).
    
    Args:
        last_positions: List of (frame_idx, x_center, y_center, bbox) tuples (last 3 positions)
        
    Returns:
        tuple: (is_hit, frame_idx, y_center, vertical_span, bbox) or (False, None, None, None, None)
    """
    if len(last_positions) != 3:
        return False, None, None, None, None
    
    (f0, x0, y0, bbox0), (f1, x1c, y1c, bbox1c), (f2, x2c, y2c, bbox2c) = last_positions
    
    # Check if middle point is a local maximum (ball at lowest point)
    going_down_then_up = (y0 < y1c) and (y2c < y1c)
    vertical_span = y1c - min(y0, y2c)
    
    if going_down_then_up and vertical_span >= MIN_VERTICAL_AMPLITUDE:
        return True, f1, y1c, vertical_span, bbox1c
    
    return False, None, None, None, None


def get_pose_keypoints(frame: np.ndarray, model_pose: YOLO) -> List[np.ndarray]:
    """
    Run YOLO-pose model to detect player keypoints in the frame.
    
    Args:
        frame: Video frame to analyze
        model_pose: YOLO pose model
        
    Returns:
        List of poses, each containing keypoints data
    """
    results = model_pose(frame, verbose=False, conf=0.3)[0]
    
    if results.keypoints is None or len(results.keypoints.data) == 0:
        return []
    
    # Extract keypoints data
    # keypoints shape: (num_persons, num_keypoints, 3) where 3 = (x, y, confidence)
    poses = []
    for person_keypoints in results.keypoints.data:
        poses.append(person_keypoints.cpu().numpy())
    
    return poses


def find_closest_player(ball_center: Tuple[float, float], poses: Sequence[np.ndarray]) -> Tuple[Optional[np.ndarray], int]:
    """
    Find the player closest to the ball.
    
    Args:
        ball_center: Tuple (x, y) of ball center
        poses: List of pose keypoints arrays
        
    Returns:
        Tuple (player_pose, player_id) or (None, -1) if no players detected
    """
    if not poses:
        return None, -1
    
    ball_x, ball_y = ball_center
    min_distance = float('inf')
    closest_player_idx = -1
    
    print(f"  [FIND_CLOSEST_PLAYER] Found {len(poses)} poses")

    for i, pose in enumerate(poses):
        # Calculate player center from valid keypoints
        valid_keypoints = pose[pose[:, 2] > 0.3]  # Filter by confidence > 0.3
        if len(valid_keypoints) == 0:
            continue
        
        player_x = np.mean(valid_keypoints[:, 0])
        player_y = np.mean(valid_keypoints[:, 1])
        
        # Calculate distance to ball
        distance = np.sqrt((ball_x - player_x)**2 + (ball_y - player_y)**2)
        
        if distance < min_distance:
            min_distance = distance
            closest_player_idx = i
    
    if closest_player_idx == -1:
        return None, -1
    
    return poses[closest_player_idx], closest_player_idx


def classify_hit_type(ball_bbox: np.ndarray, player_pose: Optional[np.ndarray]) -> str:
    """
    Classify hit type (Head/Foot/Unknown) based on ball position and player keypoints.
    
    Args:
        ball_bbox: Ball bounding box in xyxy format
        player_pose: Player keypoint array (17, 3) with (x, y, confidence)
        
    Returns:
        String: 'Head', 'Foot', or 'Unknown'
    """
    if player_pose is None:
        print("  [CLASSIFY] No player pose detected -> Unknown")
        return 'Unknown'
    
    # Calculate ball center
    ball_x = (ball_bbox[0] + ball_bbox[2]) / 2
    ball_y = (ball_bbox[1] + ball_bbox[3]) / 2

    print(f"\n  [CLASSIFY] Ball position: x={ball_x:.1f}, y={ball_y:.1f}")
    
    # COCO keypoint indices:
    # Head: 0=nose, 3=left_ear, 4=right_ear
    # Feet: 15=left_ankle, 16=right_ankle
    head_indices = [0, 3, 4]
    foot_indices = [15, 16]
    head_names = ['nose', 'left_ear', 'right_ear']
    foot_names = ['left_ankle', 'right_ankle']
    
    # Calculate distance to head keypoints
    head_distances = []
    print(f"  [CLASSIFY] Head keypoints:")
    for idx, name in zip(head_indices, head_names):
        if idx < len(player_pose) and player_pose[idx, 2] > 0.3:  # Check confidence
            kp_x, kp_y, kp_conf = player_pose[idx, 0], player_pose[idx, 1], player_pose[idx, 2]
            dist = np.sqrt((ball_x - kp_x)**2 + (ball_y - kp_y)**2)
            head_distances.append(dist)
            print(f"    - {name:12s}: pos=({kp_x:.1f}, {kp_y:.1f}), conf={kp_conf:.2f}, dist={dist:.1f}")
        else:
            conf_str = f"{player_pose[idx, 2]:.2f}" if idx < len(player_pose) else "N/A"
            print(f"    - {name:12s}: SKIPPED (conf={conf_str})")
    
    # Calculate distance to foot keypoints
    foot_distances = []
    print(f"  [CLASSIFY] Foot keypoints:")
    for idx, name in zip(foot_indices, foot_names):
        if idx < len(player_pose) and player_pose[idx, 2] > 0.3:  # Check confidence
            kp_x, kp_y, kp_conf = player_pose[idx, 0], player_pose[idx, 1], player_pose[idx, 2]
            dist = np.sqrt((ball_x - kp_x)**2 + (ball_y - kp_y)**2)
            foot_distances.append(dist)
            print(f"    - {name:12s}: pos=({kp_x:.1f}, {kp_y:.1f}), conf={kp_conf:.2f}, dist={dist:.1f}")
        else:
            conf_str = f"{player_pose[idx, 2]:.2f}" if idx < len(player_pose) else "N/A"
            print(f"    - {name:12s}: SKIPPED (conf={conf_str})")
    
    # Determine hit type based on minimum distances
    min_head_dist = min(head_distances) if head_distances else float('inf')
    min_foot_dist = min(foot_distances) if foot_distances else float('inf')
    
    print(f"  [CLASSIFY] Min distances: head={min_head_dist:.1f}, foot={min_foot_dist:.1f}")
    
    # If both are unavailable
    if min_head_dist == float('inf') and min_foot_dist == float('inf'):
        print(f"  [CLASSIFY] No valid keypoints detected -> Unknown")
        return 'Unknown'
    
    # Classification with threshold (prefer the closer one with a margin)
    distance_threshold = 80  # pixels - adjust based on video resolution
    print(f"  [CLASSIFY] Distance threshold: {distance_threshold} pixels")
    
    # Decision logic
    if min_head_dist < distance_threshold and min_head_dist < min_foot_dist:
        print(f"  [CLASSIFY] DECISION: Head (head_dist {min_head_dist:.1f} < threshold {distance_threshold} AND head_dist < foot_dist {min_foot_dist:.1f})")
        return 'Head'
    elif min_foot_dist < distance_threshold:
        print(f"  [CLASSIFY] DECISION: Foot (foot_dist {min_foot_dist:.1f} < threshold {distance_threshold})")
        return 'Foot'
    else:
        print(f"  [CLASSIFY] DECISION: Unknown (both distances exceed threshold: head={min_head_dist:.1f}, foot={min_foot_dist:.1f})")
        return 'Unknown'




def draw_debug_keypoints(frame: np.ndarray, player_pose: Optional[np.ndarray]) -> np.ndarray:
    """
    Draw small red boxes at nose, left ankle, and right ankle for debugging.
    
    Args:
        frame: Video frame to annotate
        player_pose: Player keypoint array (17, 3) with (x, y, confidence)
        
    Returns:
        Annotated frame
    """
    if player_pose is None:
        return frame
    
    # COCO keypoint indices: 0=nose, 3=left_ear, 4=right_ear, 15=left_ankle, 16=right_ankle
    keypoint_indices = [0, 3, 4, 15, 16]
    keypoint_names = ['nose', 'left_ear', 'right_ear', 'left_ankle', 'right_ankle']
    colors = [(0, 0, 255), (0, 0, 255), (0, 0, 255), (0, 0, 255), (0, 0, 255)]  # Red for all
    
    for idx, name, color in zip(keypoint_indices, keypoint_names, colors):
        if idx < len(player_pose) and player_pose[idx, 2] > 0.3:  # Check confidence
            kp_x, kp_y = int(player_pose[idx, 0]), int(player_pose[idx, 1])
            
            # Draw small red box (5x5 pixels)
            box_size = 5
            cv2.rectangle(
                frame,
                (kp_x - box_size, kp_y - box_size),
                (kp_x + box_size, kp_y + box_size),
                color,
                thickness=2
            )
            
            # Draw label next to the box
            cv2.putText(
                frame,
                name,
                (kp_x + box_size + 2, kp_y),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.4,
                color,
                1,
                cv2.LINE_AA
            )
    
    return frame

def check_and_record_hit(
    last_ball_positions: List[BallPosition],
    hit_detections: HitDetections,
    fps: float,
    min_frames_between_hits: int,
    frame: np.ndarray,
    model_pose: YOLO,
    frame_buffer: Optional[Sequence[Tuple[int, np.ndarray]]] = None,
    debug_dir: Optional[str] = None,
) -> HitDetections:
    """
    Check for hit and record it if valid, with hit type classification.
    
    Args:
        last_positions: List of (frame_idx, x_center, y_center, bbox) tuples
        hit_detections: List of hit metadata dicts
        fps: Video frames per second
        min_frames_between_hits: Minimum frames between consecutive hits
        frame: Current video frame (fallback if frame_buffer lookup fails)
        model_pose: YOLO pose model
        frame_buffer: Optional list of (frame_idx, frame_image) tuples for debug output and frame retrieval
        debug_dir: Optional directory to save debug frames
        
    Returns:
        Updated hit_detections list
    """
    is_hit, hit_n_frame, hit_y, span, hit_bbox = detect_hit(last_ball_positions)
    
    if is_hit:
        # Check minimum gap between hits
        last_hit_frame = hit_detections[-1]['frame'] if hit_detections else None
        if not hit_detections or (hit_n_frame - last_hit_frame) >= min_frames_between_hits:
            # Retrieve the correct frame from frame_buffer matching hit_n_frame
            hit_frame = frame  # fallback to current frame
            if frame_buffer is not None:
                for frame_idx, frame_image in frame_buffer:
                    if frame_idx == hit_n_frame:
                        hit_frame = frame_image
                        break
            
            # Analyze pose to classify hit type using the correct frame
            poses = get_pose_keypoints(hit_frame, model_pose)
            
            ball_center = ((hit_bbox[0] + hit_bbox[2]) / 2, (hit_bbox[1] + hit_bbox[3]) / 2)
            
            player_pose, player_id = find_closest_player(ball_center, poses)
            
            hit_type = classify_hit_type(hit_bbox, player_pose)
            
            hit_detections = hit_detections.copy()
            hit_detections.append({
                'frame': hit_n_frame,
                'type': hit_type,
                'player_id': player_id,
                'player_pose': player_pose  # Store pose for visualization
            })
            t_sec = hit_n_frame / fps
            hit_number = len(hit_detections)
            print(f"HIT #{hit_number} at frame {hit_n_frame} (t={t_sec:.2f}s), Type: {hit_type}, Player: {player_id}, y={hit_y:.1f}, span={span:.1f}")
            
            # Save debug frames if frame buffer is available
            if frame_buffer is not None and debug_dir is not None:
                save_debug_frames(frame_buffer, hit_n_frame, hit_number, debug_dir)
            
            return hit_detections
    
    return hit_detections


def annotate_frame(
    frame: np.ndarray,
    detections: sv.Detections,
    box_annotator: sv.BoxAnnotator,
    label_annotator: sv.LabelAnnotator,
    hit_detections: HitDetections,
    n_frame: int,
) -> np.ndarray:
    """
    Annotate frame with ball detection, hit counter, and debug keypoints.
    
    Args:
        frame: Video frame to annotate
        detections: sv.Detections object
        box_annotator: Supervision box annotator
        label_annotator: Supervision label annotator
        hit_detections: List of hit metadata dicts
        n_frame: Current frame number
        
    Returns:
        Annotated frame
    """
    annotated_frame = frame.copy()
    
    # Draw ball detection box and label
    if len(detections) > 0:
        conf = float(detections.confidence[0])
        labels = [f"Ball {conf:.2f}"]
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
        )
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels,
        )
    
    # Draw debug keypoints for the most recent hit (show for 10 frames after hit)
    if hit_detections:
        last_hit = hit_detections[-1]
        frames_since_hit = n_frame - last_hit['frame']
        if 0 <= frames_since_hit <= 10 and 'player_pose' in last_hit:
            annotated_frame = draw_debug_keypoints(annotated_frame, last_hit['player_pose'])
    
    # Draw hit counter HUD
    annotated_frame = draw_hit_counter(annotated_frame, hit_detections)
    
    return annotated_frame


def draw_hit_counter(frame: np.ndarray, hit_detections: Sequence[Dict[str, object]]) -> np.ndarray:
    """
    Draw a hit counter HUD on the frame with breakdown by hit type.
    
    Args:
        frame: Video frame to annotate
        hit_detections: List of hit metadata dicts
        
    Returns:
        Annotated frame
    """
    # Calculate hit counts by type
    total_hits = len(hit_detections)
    head_hits = sum(1 for h in hit_detections if h['type'] == 'Head')
    foot_hits = sum(1 for h in hit_detections if h['type'] == 'Foot')
    unknown_hits = sum(1 for h in hit_detections if h['type'] == 'Unknown')
    
    hit_text = f"Hits: {total_hits} | Head: {head_hits} | Foot: {foot_hits} | Unknown: {unknown_hits}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    
    # Measure text size
    (text_width, text_height), baseline = cv2.getTextSize(
        hit_text, font, font_scale, thickness
    )
    
    # Box position and padding
    pad_x, pad_y = 10, 10
    x1, y1 = 10, 10
    x2 = x1 + text_width + 2 * pad_x
    y2 = y1 + text_height + 2 * pad_y
    
    # Draw filled rectangle
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), thickness=-1)
    
    # Draw text
    text_x = x1 + pad_x
    text_y = y1 + pad_y + text_height
    cv2.putText(
        frame, hit_text, (text_x, text_y),
        font, font_scale, (0, 255, 0), thickness, cv2.LINE_AA
    )
    
    return frame


def save_debug_frames(
    frame_buffer: Sequence[Tuple[int, np.ndarray]],
    hit_frame: int,
    hit_number: int,
    debug_dir: str,
) -> None:
    """
    Save the 3 frames around a detected hit for debugging.
    
    Args:
        frame_buffer: List of (frame_idx, frame_image) tuples
        hit_frame: Frame number where the hit occurred
        hit_number: Sequential hit number (1-indexed)
        debug_dir: Directory to save debug frames
    """
    import os
    
    # Create debug directory if it doesn't exist
    os.makedirs(debug_dir, exist_ok=True)
    
    # Find frames to save: hit_frame-1, hit_frame, hit_frame+1
    frames_to_save = [hit_frame - 1, hit_frame, hit_frame + 1]
    
    for frame_idx, frame_image in frame_buffer:
        if frame_idx in frames_to_save:
            filename = f"hit-{hit_number}-frame-{frame_idx}.png"
            filepath = os.path.join(debug_dir, filename)
            cv2.imwrite(filepath, frame_image)
            print(f"  💾 Saved debug frame: {filename}")



In [6]:
def debug_print_detections(detections: sv.Detections, n_frame: int) -> None:
    if len(detections) > 0:
        conf = float(detections.confidence[0])
        x1, y1, x2, y2 = detections.xyxy[0].tolist()

        print(
            f"FRAME {n_frame:4d}: best conf={conf:.3f}, "
            f"bbox=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})"
        )
    else:
        print(f"FRAME {n_frame:4d}: no detection")


## Process Video

Run ball detection and hit counting on the video.


In [ ]:
# Get video info
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
fps = video_info.fps
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"Processing {VIDEO_PATH}")
print(f"FPS: {fps}, Resolution: {video_info.width}x{video_info.height}\n")

# Initialize tracking state
last_ball_positions = []  # Track last 3 (frame_idx, x_center, y_center, bbox) positions
hit_detections = []  # List of hit metadata dicts: {'frame': int, 'type': str, 'player_id': int}
last_ball_detection_n_frame = None  # Last frame with a detection
frame_buffer = []  # Buffer to keep last 3 frames for debug output
previous_frame = None

# Detection statistics
processed_frames = 0
frames_with_ball_detection = 0

# Process video
with sv.VideoSink(target_path=OUTPUT_PATH, video_info=video_info) as sink:
    for n_frame, frame in enumerate(frames_generator, start=1):

        processed_frames += 1

        if previous_frame is None:
            previouse_frame = frame

        # Add current frame to buffer (keep last 3 frames)
        frame_buffer.append((n_frame, frame.copy()))
        if len(frame_buffer) > 3:
            frame_buffer.pop(0)

        # Run YOLO detection
        results = model(
            frame,
            verbose=False,
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_NMS,
        )[0]

        ball_detections = sv.Detections.from_ultralytics(results)

        # Filter to keep only best detection
        ball_detections = filter_best_ball_detection(ball_detections, MIN_CONFIDENCE)

        debug_print_detections(ball_detections, n_frame)

        if len(ball_detections) > 0:
            frames_with_ball_detection += 1

        # Update tracking state
        last_ball_positions, last_ball_detection_n_frame = update_ball_tracking_state(
            ball_detections,
            n_frame,
            last_ball_positions,
            last_ball_detection_n_frame,
            GAP_RESET_FRAMES
        )

        # Check for hit and record it
        if len(ball_detections) > 0:
            hit_detections = check_and_record_hit(
                last_ball_positions,
                hit_detections,
                fps,
                MIN_FRAMES_BETWEEN_HITS,
                previous_frame,
                model_pose,
                frame_buffer=frame_buffer,
                debug_dir=DEBUG_FRAMES_DIR
            )

        # Annotate frame with detections and hit counter
        annotated_frame = annotate_frame(
            frame, ball_detections, box_annotator, label_annotator, hit_detections, n_frame
        )

        # Write frame to output video
        sink.write_frame(annotated_frame)

        previous_frame = frame

detection_percentage = frames_with_ball_detection / processed_frames if processed_frames > 0 else 0.0

print(f"Done! Video saved to {OUTPUT_PATH}")
print(f"📊 Total hits detected: {len(hit_detections)}")
print(f"⚽ Ball detection percentage: {detection_percentage:.3f}")
print(f"🐛 Debug frames saved to: {DEBUG_FRAMES_DIR}")

video_filename = os.path.basename(VIDEO_PATH)
json_output_path = os.path.join(
    os.path.dirname(OUTPUT_PATH) or ".",
    os.path.splitext(video_filename)[0] + ".json"
)

summary_data = {
    "video_name": video_filename,
    "ball_detection_percentage": detection_percentage
}

with open(json_output_path, "w") as json_file:
    json.dump(summary_data, json_file, indent=4)

print(f"Detection summary saved to {json_output_path}")


Processing /kaggle/input/alta-videos/altinha-beach-green-mq-13s.mp4
FPS: 30, Resolution: 852x480

FRAME    1: best conf=0.945, bbox=(453.5,73.6,478.6,99.0)
FRAME    2: best conf=0.954, bbox=(433.3,73.0,458.6,98.2)
FRAME    3: best conf=0.912, bbox=(415.3,73.1,438.5,98.4)
FRAME    4: best conf=0.941, bbox=(392.9,73.6,418.4,99.5)
FRAME    5: best conf=0.951, bbox=(372.7,75.7,397.8,100.9)
FRAME    6: best conf=0.798, bbox=(353.7,79.4,377.8,104.0)
FRAME    7: best conf=0.790, bbox=(330.0,82.3,357.6,109.4)
FRAME    8: best conf=0.900, bbox=(313.3,87.7,338.6,113.9)
FRAME    9: best conf=0.837, bbox=(292.1,94.6,318.4,118.1)
FRAME   10: best conf=0.934, bbox=(273.4,100.2,298.4,123.9)
FRAME   11: best conf=0.899, bbox=(254.2,107.3,278.3,131.3)
FRAME   12: best conf=0.903, bbox=(231.7,116.6,258.8,142.2)
FRAME   13: best conf=0.933, bbox=(212.9,126.9,239.0,150.6)
FRAME   14: best conf=0.941, bbox=(194.3,138.1,219.9,163.4)
FRAME   15: best conf=0.932, bbox=(175.0,151.0,201.0,177.1)
FRAME   16: bes

## Results Summary

Display detailed results of hit detection.


In [8]:
print("=" * 60)
print("HIT DETECTION RESULTS")
print("=" * 60)

# Calculate hit statistics
total_hits = len(hit_detections)
head_hits = sum(1 for h in hit_detections if h['type'] == 'Head')
foot_hits = sum(1 for h in hit_detections if h['type'] == 'Foot')
unknown_hits = sum(1 for h in hit_detections if h['type'] == 'Unknown')

print(f"Total hits/passes detected: {total_hits}")
print(f"  - Head hits: {head_hits} ({100*head_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Head hits: 0")
print(f"  - Foot hits: {foot_hits} ({100*foot_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Foot hits: 0")
print(f"  - Unknown: {unknown_hits} ({100*unknown_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Unknown: 0")

print(f"\nDetailed timestamps:")
print("-" * 60)

for i, hit_data in enumerate(hit_detections, start=1):
    frame_idx = hit_data['frame']
    hit_type = hit_data['type']
    player_id = hit_data['player_id']
    timestamp = frame_idx / fps
    minutes = int(timestamp // 60)
    seconds = timestamp % 60
    print(f"  Hit #{i:2d} | Frame {frame_idx:4d} | {minutes:02d}:{seconds:05.2f} | {hit_type:7s} | Player {player_id}")

print("=" * 60)


HIT DETECTION RESULTS
Total hits/passes detected: 12
  - Head hits: 8 (66.7%)
  - Foot hits: 4 (33.3%)
  - Unknown: 0 (0.0%)

Detailed timestamps:
------------------------------------------------------------
  Hit # 1 | Frame   19 | 00:00.63 | Head    | Player 1
  Hit # 2 | Frame   57 | 00:01.90 | Head    | Player 0
  Hit # 3 | Frame   96 | 00:03.20 | Foot    | Player 3
  Hit # 4 | Frame  126 | 00:04.20 | Head    | Player 2
  Hit # 5 | Frame  149 | 00:04.97 | Head    | Player 2
  Hit # 6 | Frame  165 | 00:05.50 | Head    | Player 1
  Hit # 7 | Frame  204 | 00:06.80 | Head    | Player 0
  Hit # 8 | Frame  232 | 00:07.73 | Foot    | Player 2
  Hit # 9 | Frame  274 | 00:09.13 | Head    | Player 1
  Hit #10 | Frame  297 | 00:09.90 | Foot    | Player 2
  Hit #11 | Frame  343 | 00:11.43 | Head    | Player 0
  Hit #12 | Frame  386 | 00:12.87 | Foot    | Player 1
